# Tutorial 5 – Reconstruct and Refine a GEM from KEGG

From the tutorial file:

> This exercise is about creating a model from KEGG, based on protein sequences in a FASTA file, and doing some functionality checks on the model. The example case is for the yeast Saccharomyces cerevisiae. This tutorial is more of a showcase and its main purpose is to serve as a scaffold to reconstruct a GEM for any organism.
> Open tutorial5.m in MATLAB to begin this exercise.



In [255]:
clear;
setRavenSolver('gurobi');
addpath('./source');
tolerance = 10^-7;

## 1. Metabolic reconstruction from KEEG and protein homology

Check the wiki [Use Pre Trained HMMs](https://github.com/SysBioChalmers/RAVEN/wiki/Use-Pre-Trained-HMMs) for more details.

The tutorial uses:

- [getKEGGModelForOrganism](https://sysbiochalmers.github.io/RAVEN/doc/external/kegg/getKEGGModelForOrganism.html)

Reconstructs a genome-scale metabolic model based on protein homology to the orthologies in KEGG. If the target species is not available in KEGG, the user must select a closely related species. It is also possible to circumvent protein homology search (see fastaFile parameter for more details)

❗ I cannot execute the function in the Notebook. I got this error message:

```shell
Error connecting to MATLAB. Check the status of MATLAB by clicking the "Open MATLAB" button. Retry after ensuring MATLAB is running successfully
```

Seems there is an issue with the MATLAB Kernel and calls that take long to complete; So, I excecuted the following call in console, and saved the output:

```MATLAB
model= getKEGGModelForOrganism('sce', ...
    './tutorial_data/sce.fa', ... % fastaFile
    'euk90_kegg105', ... % Dataset: <phylogeny><% CD-HIT Identity>_kegg<release>
    './output/_ignore_tutorial_5', ... % outDir
    false, ... % keepSpontaneous. Label "spontaneous".
    false, ... % keepUndefinedStoich. "n A <=> n+1 A" form.
    false, ... % keepIncomplete. Label "incomplete", "erroneous" or "unclear".
    false, ... % keepGeneral. Label "general reaction". WARNING: this script cannot remove all.
    10^-30, ... % cutOff. significance score from HMMer needed to assign genes to a KO
    0.8, ... % minScoreRatioKO
    0.3, ... % minScoreRatioG
    -1); % maxPhylDist
save('./output/tutorial_5_model.mat', 'model');
```

In [256]:
model = load('./output/tutorial_5_model.mat').model;
disp(model);

             id: 'sce'
           name: 'Automatically generated from KEGG database'
           rxns: {1589x1 cell}
       rxnNames: {1589x1 cell}
        eccodes: {1589x1 cell}
     subSystems: {1589x1 cell}
     rxnMiriams: {1589x1 cell}
       rxnNotes: {1589x1 cell}
              S: [1600x1589 double]
           mets: {1600x1 cell}
            rev: [1589x1 double]
             ub: [1589x1 double]
             lb: [1589x1 double]
              c: [1589x1 double]
              b: [1600x1 double]
          genes: {836x1 cell}
     rxnGeneMat: [1589x836 double]
       metNames: {1600x1 cell}
    metFormulas: {1600x1 cell}
         inchis: {1600x1 cell}
     metMiriams: {1600x1 cell}
          comps: {'s'}
      compNames: {'System'}
       metComps: [1600x1 double]
        grRules: {1589x1 cell}



## 2. Validating mass balance

❗ Redox and charge balance can be very tricky since formulas depends on conditions such as pH.

In [257]:
% It is not neccesary, but I preffer tables to display information.
nonBalancedReactionsTable = makeUnablancedMetabolitesTable(model);
head(nonBalancedReactionsTable)

Potential problematic reactions: 615.

              balanceStatus    C    N    O    S    P    H     F    Cl    Fe    Se    R
              _____________    _    _    _    _    _    __    _    __    __    __    _

    R00021         -1          0    0    0    0    0     0    0    0     0     0     0
    R00025          0          0    0    0    0    0    -1    0    0     0     0     0
    R00093          0          0    0    0    0    0    -1    0    0     0     0     0
    R00094          0          0    0    0    0    0    -1    0    0     0     0     0
    R00100         -1          0    0    0    0    0     1    0    0     0     0     0
    R00111          0          0    0    0    0    0     1    0    0     0     0     0
    R00114          0          0    0    0    0    0    -1    0    0     0     0     0
    R00115          0          0    0    0    0    0    -1    0    0     0     0     0



❗ to use makeSomething and consumeSomething functions, it is recomendable to ensure that there is not open exchange reactions:

In [258]:
[~, exchangeRxnsIndexes] = getExchangeRxns(model);
fprintf('There are %i exchange reactions:\n', length(exchangeRxnsIndexes));
disp(table(model.rxns(exchangeRxnsIndexes), ...
    model.lb(exchangeRxnsIndexes), ...
    model.ub(exchangeRxnsIndexes), ...
    'VariableNames', {'ID', 'LB', 'UB'}));

There are 0 exchange reactions:


### 2.1. Free production


- [makeSomething](https://sysbiochalmers.github.io/RAVEN/doc/core/makeSomething.html)

```MATLAB
function [solution, metabolite] = makeSomething(model, ignoreMets, isNames, minNrFluxes, allowExcretion, params, ignoreIntBounds)
```
Tries to excrete any metabolite using as few reactions as possible. The intended use is when you want to make sure that you model cannot synthesize anything from nothing. It is then a faster way than to use checkProduction or canProduce

In [259]:
% First, check reactions producing species from nothing:
% From the nonBalancedReactionsTable table we already known there are issues with H+. Ignore it to avoid noise.
ignoreMets = {'H+'};
[solutionFlux, metaboliteIndex] = makeSomething(model, ignoreMets, true);
metaboliteName = model.metNames(metaboliteIndex);
reactionsWithFluxIndexes = find(abs(solutionFlux) >= tolerance);
fprintf('There are %i reactions with flux. The involved metabolite is %s', ...
    length(reactionsWithFluxIndexes), metaboliteName{:});

There are 35 reactions with flux. The involved metabolite is H2O

In [260]:
% Assess which of these reactions have problems with their equations
% Get the reactions that have flux AND appears in the table of problematic reactions
carryingFluxProblematicReactions = intersect(...
    nonBalancedReactionsTable.Properties.RowNames, ...
    model.rxns(reactionsWithFluxIndexes));
reactions2check = nonBalancedReactionsTable(carryingFluxProblematicReactions, :);
fprintf('Reactions to check: %i\n', height(reactions2check));
disp(reactions2check);

Reactions to check: 7
              balanceStatus    C     N    O     S    P     H     F    Cl    Fe    Se    R
              _____________    __    _    __    _    _    ___    _    __    __    __    _

    R00941          0           0    0     0    0    0     -1    0    0     0     0     0
    R01058          0           0    0     0    0    0     -1    0    0     0     0     0
    R01061          0           0    0     0    0    0     -1    0    0     0     0     0
    R01221          0           0    0     0    0    0     -1    0    0     0     0     0
    R01655          0           0    0     0    0    0     -1    0    0     0     0     0
    R02110          0          -6    0    -5    0    0    -10    0    0     0     0     0
    R02413          0           0    0     0    0    0     -1    0    0     0     0     0



❗ R02110 reaction looks very suspicious, C H O are unbalanced.

In [261]:
% Print fluxes from the problematic reactions
flux2showIndexes = getIndexes(model, carryingFluxProblematicReactions, 'rxns');
flux2show = zeros(size(solutionFlux));
flux2show(flux2showIndexes) = solutionFlux(flux2showIndexes);
printFluxes(model, flux2show, false, [], [], ...
    '▪ %flux\n%rxnID (%rxnName):\n\t%eqn\n');

FLUXES:
▪ -0.085106
R00941 (10-formyltetrahydrofolate:NADP+ oxidoreductase):
	H2O[s] + NADP+[s] + 10-Formyltetrahydrofolate[s] <=> NADPH[s] + CO2[s] + H+[s] + Tetrahydrofolate[s]
▪ 0.42553
R01058 (D-glyceraldehyde 3-phosphate:NADP+ oxidoreductase):
	H2O[s] + NADP+[s] + D-Glyceraldehyde 3-phosphate[s] <=> NADPH[s] + H+[s] + 3-Phospho-D-glycerate[s]
▪ 0.042553
R01061 (D-glyceraldehyde-3-phosphate:NAD+ oxidoreductase (phosphorylating)):
	NAD+[s] + Orthophosphate[s] + D-Glyceraldehyde 3-phosphate[s] <=> NADH[s] + H+[s] + 3-Phospho-D-glyceroyl phosphate[s]
▪ -0.042553
R01221 (glycine:NAD+ 2-oxidoreductase (tetrahydrofolate-methylene-adding);):
	NAD+[s] + Glycine[s] + Tetrahydrofolate[s] <=> NADH[s] + CO2[s] + Ammonia[s] + H+[s] + 5,10-Methylenetetrahydrofolate[s]
▪ -0.085106
R01655 (5,10-Methenyltetrahydrofolate 5-hydrolase (decyclizing)):
	H2O[s] + 5,10-Methenyltetrahydrofolate[s] <=> H+[s] + 10-Formyltetrahydrofolate[s]
▪ 0.40426
R02110 (1,4-alpha-D-Glucan:1,4-alpha-D-glucan 6-alpha-D-(1,

The reaction involves two polysacharides. I want to remark the following from [KEEG COMPOUND Database](https://www.genome.jp/kegg/compound/):

> "**Biosynthetic Codes**<br>
> The structures of DNA, RNA, and proteins are determined by template-based syntheses of replication, transcription, and translation with the genetic code. In contrast, the structures of glycans, lipids, polyketides, nonribosomal peptides, and various secondary metabolites are determined by biosynthetic pathways. KEGG COMPOUND, as well as KEGG GLYCAN, is a resource for understanding such biosynthetic codes and for inferring chemical repertoires of these diverse substances from genomic information."

❗ R02110 is not an interconversión but a polymerization reaction, adding a glucose polymer Amylose (C6H10O5)n to another glucose polymer Starch (C12H20O10)n:

From [KEGG GLYCAN Database](https://www.genome.jp/kegg/glycan/), R02110 is the same as R06186:

<img src="https://www.genome.jp/Fig/reaction/R06186.gif"/>

In [262]:
% Looking for al starch reactions
printFluxes(model, solutionFlux, false, [], [], ...
    '▪ %flux\n%rxnID (%rxnName):\n\t%eqn\n', {'Starch'});

FLUXES:
▪ 0.40426
R02110 (1,4-alpha-D-Glucan:1,4-alpha-D-glucan 6-alpha-D-(1,4-alpha-D-glucano)-transferase):
	Amylose[s] <=> Starch[s]
▪ 0.40426
R02111 (1,4-alpha-D-Glucan:orthophosphate alpha-D-glucosyltransferase):
	Orthophosphate[s] + Starch[s] <=> D-Glucose 1-phosphate[s] + Amylose[s]


❗ Why is there Starch in a yeast in first place? 🤔

The tutorial removes only R02110, but, unless it is a genetically engineered yeast, I will consider to remove also R02111. 


In [263]:
model=removeReactions(model,'R02110');

In [264]:
[solutionFlux, metaboliteIndex] = makeSomething(model, ignoreMets, true);
metaboliteName = model.metNames(metaboliteIndex);
reactionsWithFluxIndexes = find(abs(solutionFlux) >= tolerance);
fprintf('There are %i reactions with flux. The involved metabolite is %s', ...
    length(reactionsWithFluxIndexes), metaboliteName{:});

There are 0 reactions with flux. The involved metabolite is 

### 2.2. Free internal consumption

- [consumeSomething](https://sysbiochalmers.github.io/RAVEN/doc/core/consumeSomething.html)

```MATLAB
function [solution, metabolite] = consumeSomething(model, ignoreMets, isNames, minNrFluxes, params, ignoreIntBounds)
```
Tries to consume any metabolite using as few reactions as possible. The intended use is when you want to make sure that you model cannot consume anything without producing something. It is intended to be used with no active exchange reactions.


In [ ]:
% something without any output?
[solutionFlux, metaboliteIndex] = consumeSomething(model,{'H+'},true);